# Weaviate

This guide will help you getting started with such a retriever backed by a [Weaviate vector store](/docs/integrations/vectorstores/weaviate). For detailed documentation of all features and configurations head to the [API reference](https://api.js.langchain.com/classes/langchain.retrievers_self_query.SelfQueryRetriever.html).

## Overview

A [self-query retriever](/docs/how_to/self_query/) retrieves documents by dynamically generating metadata filters based on some input query. This allows the retriever to account for underlying document metadata in addition to pure semantic similarity when fetching results.

It uses a module called a `Translator` that generates a filter based on information about metadata fields and the query language that a given vector store supports.

### Integration details

| Backing vector store | Self-host | Cloud offering | Package | [Py support](https://python.langchain.com/docs/integrations/retrievers/self_query/weaviate_self_query/) |
| :--- | :--- | :---: | :---: | :---: |
[`WeaviateVectorStore`](https://api.js.langchain.com/classes/langchain_weaviate.WeaviateStore.html) | ✅ | ✅ | [`@langchain/weaviate`](https://www.npmjs.com/package/@langchain/weaviate) | ✅ |

## Setup

Set up a Weaviate instance as documented [here](/docs/integrations/vectorstores/weaviate). Set the following environment variables if relevant:

```ts
// Include port if relevant, e.g. "localhost:8080"
process.env.WEAVIATE_URL = "YOUR_WEAVIATE_URL";
// Optional, for cloud deployments
process.env.WEAVIATE_API_KEY = "YOUR_API_KEY";
```

If you want to get automated tracing from individual queries, you can also set your [LangSmith](https://docs.smith.langchain.com/) API key by uncommenting below:

```typescript
// process.env.LANGSMITH_API_KEY = "<YOUR API KEY HERE>";
// process.env.LANGSMITH_TRACING = "true";
```

### Installation

The vector store lives in the `@langchain/weaviate` package. You'll also need to install the `langchain` package to import the main `SelfQueryRetriever` class.

The official Weaviate SDK (`weaviate-client`) is automatically installed as a dependency of `@langchain/weaviate`, but you may wish to install it independently as well.

For this example, we'll also use OpenAI embeddings, so you'll need to install the `@langchain/openai` package and [obtain an API key](https://platform.openai.com):

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/weaviate langchain @langchain/openai @langchain/core weaviate-client
</Npm2Yarn>
```

## Instantiation

First, initialize your Weaviate vector store with some documents that contain metadata:

In [ ]:
import { OpenAIEmbeddings } from "@langchain/openai";
import { WeaviateStore } from "@langchain/weaviate";
import { Document } from "@langchain/core/documents";
import type { AttributeInfo } from "langchain/chains/query_constructor";

import weaviate from "weaviate-client";

/**
 * First, we create a bunch of documents. You can load your own documents here instead.
 * Each document has a pageContent and a metadata field. Make sure your metadata matches the AttributeInfo below.
 */
const docs = [
  new Document({
    pageContent:
      "A bunch of scientists bring back dinosaurs and mayhem breaks loose",
    metadata: { year: 1993, rating: 7.7, genre: "science fiction" },
  }),
  new Document({
    pageContent:
      "Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
    metadata: { year: 2010, director: "Christopher Nolan", rating: 8.2 },
  }),
  new Document({
    pageContent:
      "A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
    metadata: { year: 2006, director: "Satoshi Kon", rating: 8.6 },
  }),
  new Document({
    pageContent:
      "A bunch of normal-sized women are supremely wholesome and some men pine after them",
    metadata: { year: 2019, director: "Greta Gerwig", rating: 8.3 },
  }),
  new Document({
    pageContent: "Toys come alive and have a blast doing so",
    metadata: { year: 1995, genre: "animated" },
  }),
  new Document({
    pageContent: "Three men walk into the Zone, three men walk out of the Zone",
    metadata: {
      year: 1979,
      director: "Andrei Tarkovsky",
      genre: "science fiction",
      rating: 9.9,
    },
  }),
];

/**
 * Next, we define the attributes we want to be able to query on.
 * in this case, we want to be able to query on the genre, year, director, rating, and length of the movie.
 * We also provide a description of each attribute and the type of the attribute.
 * This is used to generate the query prompts.
 */
const attributeInfo: AttributeInfo[] = [
  {
    name: "genre",
    description: "The genre of the movie",
    type: "string or array of strings",
  },
  {
    name: "year",
    description: "The year the movie was released",
    type: "number",
  },
  {
    name: "director",
    description: "The director of the movie",
    type: "string",
  },
  {
    name: "rating",
    description: "The rating of the movie (1-10)",
    type: "number",
  },
  {
    name: "length",
    description: "The length of the movie in minutes",
    type: "number",
  },
];

/**
 * Next, we instantiate a vector store. This is where we store the embeddings of the documents.
 * We also need to provide an embeddings object. This is used to embed the documents.
 */
const client = weaviate.connectToWeaviateCloud({
   clusterURL: process.env.WEAVIATE_URL!, 
	 options : {
      authCredentials: new weaviate.ApiKey(process.env.WEAVIATE_API_KEY || "")
    },
});

const embeddings = new OpenAIEmbeddings();
const vectorStore = await WeaviateStore.fromDocuments(docs, embeddings, {
  client,
  indexName: "Test",
  textKey: "text",
  metadataKeys: ["year", "director", "rating", "genre"],
});

Now we can instantiate our retriever:

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```

In [2]:
// @lc-docs-hide-cell

import { ChatOpenAI } from "@langchain/openai";

const llm = new ChatOpenAI({
  model: "gpt-4o",
  temperature: 0,
});

In [3]:
import { SelfQueryRetriever } from "langchain/retrievers/self_query";
import { WeaviateTranslator } from "@langchain/weaviate";

const selfQueryRetriever = SelfQueryRetriever.fromLLM({
  llm: llm,
  vectorStore: vectorStore,
  /** A short summary of what the document contents represent. */
  documentContents: "Brief summary of a movie",
  attributeInfo: attributeInfo,
  structuredQueryTranslator: new WeaviateTranslator(),
});

## Usage

Now, ask a question that requires some knowledge of the document's metadata to answer. You can see that the retriever will generate the correct result:

In [4]:
await selfQueryRetriever.invoke(
  "Which movies are rated higher than 8.5?"
);

[
  Document {
    pageContent: 'A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea',
    metadata: { director: 'Satoshi Kon', genre: null, rating: 8.6, year: 2006 },
    id: undefined
  },
  Document {
    pageContent: 'Three men walk into the Zone, three men walk out of the Zone',
    metadata: {
      director: 'Andrei Tarkovsky',
      genre: 'science fiction',
      rating: 9.9,
      year: 1979
    },
    id: undefined
  }
]


## Use within a chain

Like other retrievers, Weaviate self-query retrievers can be incorporated into LLM applications via [chains](/docs/how_to/sequence/).

Note that because their returned answers can heavily depend on document metadata, we format the retrieved documents differently to include that information.

In [5]:
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { RunnablePassthrough, RunnableSequence } from "@langchain/core/runnables";
import { StringOutputParser } from "@langchain/core/output_parsers";

import type { Document } from "@langchain/core/documents";

const prompt = ChatPromptTemplate.fromTemplate(`
Answer the question based only on the context provided.

Context: {context}

Question: {question}`);

const formatDocs = (docs: Document[]) => {
  return docs.map((doc) => JSON.stringify(doc)).join("\n\n");
}

// See https://js.langchain.com/docs/tutorials/rag
const ragChain = RunnableSequence.from([
  {
    context: selfQueryRetriever.pipe(formatDocs),
    question: new RunnablePassthrough(),
  },
  prompt,
  llm,
  new StringOutputParser(),
]);

In [6]:
await ragChain.invoke("Which movies are rated higher than 8.5?");

Both movies are rated higher than 8.5. The first movie directed by Satoshi Kon has a rating of 8.6, and the second movie directed by Andrei Tarkovsky has a rating of 9.9.


## Default search params

You can also pass a `searchParams` field into the above method that provides default filters applied in addition to any generated query.

In [7]:
const selfQueryRetrieverWithDefaultParams = SelfQueryRetriever.fromLLM({
  llm: llm,
  vectorStore: vectorStore,
  documentContents: "Brief summary of a movie",
  attributeInfo: attributeInfo,
  structuredQueryTranslator: new WeaviateTranslator(),
  searchParams: {
    filter: {
      where: {
        operator: "Equal",
        path: ["type"],
        valueText: "movie",
      },
    },
    mergeFiltersOperator: "or",
  },
});

## API reference

For detailed documentation of all Weaviate self-query retriever features and configurations head to the [API reference](https://api.js.langchain.com/classes/langchain.retrievers_self_query.SelfQueryRetriever.html).